In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
def bert_module(query, key, value, embed_dim, num_head, i):
    
    # Multi headed self-attention
    attention_output = layers.MultiHeadAttention(
        num_heads=num_head,
        key_dim=embed_dim // num_head,
        name="encoder_{}/multiheadattention".format(i)
    )(query, key, value, use_causal_mask=True)
    
    # Add & Normalize
    attention_output = layers.Add()([query, attention_output])  # Skip Connection
    attention_output = layers.LayerNormalization(epsilon=1e-6)(attention_output)
    
    # Feedforward network
    ff_net = keras.models.Sequential([
        layers.Dense(2 * embed_dim, activation='relu', name="encoder_{}/ffn_dense_1".format(i)),
        layers.Dense(embed_dim, name="encoder_{}/ffn_dense_2".format(i)),
    ])

    # Apply Feedforward network
    ffn_output = ff_net(attention_output)

    # Add & Normalize
    ffn_output = layers.Add()([attention_output, ffn_output])  # Skip Connection
    ffn_output = layers.LayerNormalization(epsilon=1e-6)(ffn_output)
    
    return ffn_output

In [5]:
def get_sinusoidal_embeddings(sequence_length, embedding_dim):
    position_enc = np.array([
        [pos / np.power(10000, 2. * i / embedding_dim) for i in range(embedding_dim)]
        if pos != 0 else np.zeros(embedding_dim)
        for pos in range(sequence_length)
    ])
    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2])  # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2])  # dim 2i+1
    return tf.cast(position_enc, dtype=tf.float32)

In [7]:
N = 20 # vocab_size

vocabs = ['word_' + str(i) for i in range(N)]

vocab_map = {}
for i in range(len(vocabs)):
    vocab_map[vocabs[i]] = i
    
pairs = []

for i in vocabs:
    for j in vocabs:
        for k in vocabs:
            if i != j and i != k and j != k:
                pairs.append((i,j,k))
            
#indicator = np.random.choice([0, 1], size=len(pairs), p=[0.5, 0.5])

# pairs_train = [pairs[i] for i in range(len(indicator)) if indicator[i] == 1]
# pairs_test = [pairs[i] for i in range(len(indicator)) if indicator[i] == 0]

pairs_train = [x for x in pairs if int(x[0].split('_')[-1]) <= 9]
pairs_test = [x for x in pairs if int(x[0].split('_')[-1]) >= 10]

In [8]:
sentences_train = []
sentences_number_train = []
sentences_test = []
sentences_number_test = []

for pair in pairs_train:
    sentences_train.append([pair[0], pair[1], pair[2], pair[0]])
    sentences_number_train.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
for pair in pairs_test:
    sentences_test.append([pair[0], pair[1], pair[2], pair[0]])
    sentences_number_test.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
x_masked_train = []
y_masked_labels_train = []
x_masked_test = []
y_masked_labels_test = []

for pair in pairs_train:
    x_masked_train.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    y_masked_labels_train.append([vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
for pair in pairs_test:
    x_masked_test.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    y_masked_labels_test.append([vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
x_masked_train = np.array(x_masked_train)
y_masked_labels_train = np.array(y_masked_labels_train)
x_masked_test = np.array(x_masked_test)
y_masked_labels_test = np.array(y_masked_labels_test)

perm = np.random.permutation(len(x_masked_train))
x_masked_train = x_masked_train[perm]
y_masked_labels_train = y_masked_labels_train[perm]

In [9]:
embed_dim = 10
num_head = 2

callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
word_embeddings = layers.Embedding(N, embed_dim, name="word_embedding")(inputs)
encoder_output = word_embeddings

for i in range(1):
    encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
mlm_output = layers.Dense(N, name="mlm_cls", activation="softmax")(encoder_output)
mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
adam = Adam()
mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=2000, batch_size=5000, 
                        verbose=2)

2024-05-22 07:06:20.584513: W tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-05-22 07:06:20.584555: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-05-22 07:06:20.584576: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gl3035.arc-ts.umich.edu): /proc/driver/nvidia/version does not exist
2024-05-22 07:06:20.584787: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/2000
1/1 - 1s - loss: 3.3615 - val_loss: 3.3031 - 1s/epoch - 1s/step
Epoch 2/2000
1/1 - 0s - loss: 3.3175 - val_loss: 3.2610 - 54ms/epoch - 54ms/step
Epoch 3/2000
1/1 - 0s - loss: 3.2739 - val_loss: 3.2230 - 49ms/epoch - 49ms/step
Epoch 4/2000
1/1 - 0s - loss: 3.2332 - val_loss: 3.1907 - 53ms/epoch - 53ms/step
Epoch 5/2000
1/1 - 0s - loss: 3.1976 - val_loss: 3.1645 - 62ms/epoch - 62ms/step
Epoch 6/2000
1/1 - 0s - loss: 3.1674 - val_loss: 3.1434 - 57ms/epoch - 57ms/step
Epoch 7/2000
1/1 - 0s - loss: 3.1424 - val_loss: 3.1264 - 53ms/epoch - 53ms/step
Epoch 8/2000
1/1 - 0s - loss: 3.1220 - val_loss: 3.1128 - 62ms/epoch - 62ms/step
Epoch 9/2000
1/1 - 0s - loss: 3.1053 - val_loss: 3.1016 - 53ms/epoch - 53ms/step
Epoch 10/2000
1/1 - 0s - loss: 3.0915 - val_loss: 3.0921 - 79ms/epoch - 79ms/step
Epoch 11/2000
1/1 - 0s - loss: 3.0798 - val_loss: 3.0836 - 60ms/epoch - 60ms/step
Epoch 12/2000
1/1 - 0s - loss: 3.0698 - val_loss: 3.0760 - 64ms/epoch - 64ms/step
Epoch 13/2000
1/1 - 0s - loss

1/1 - 0s - loss: 2.8634 - val_loss: 2.8959 - 80ms/epoch - 80ms/step
Epoch 102/2000
1/1 - 0s - loss: 2.8620 - val_loss: 2.8949 - 80ms/epoch - 80ms/step
Epoch 103/2000
1/1 - 0s - loss: 2.8606 - val_loss: 2.8938 - 59ms/epoch - 59ms/step
Epoch 104/2000
1/1 - 0s - loss: 2.8592 - val_loss: 2.8928 - 49ms/epoch - 49ms/step
Epoch 105/2000
1/1 - 0s - loss: 2.8579 - val_loss: 2.8918 - 51ms/epoch - 51ms/step
Epoch 106/2000
1/1 - 0s - loss: 2.8566 - val_loss: 2.8908 - 62ms/epoch - 62ms/step
Epoch 107/2000
1/1 - 0s - loss: 2.8552 - val_loss: 2.8898 - 62ms/epoch - 62ms/step
Epoch 108/2000
1/1 - 0s - loss: 2.8539 - val_loss: 2.8888 - 52ms/epoch - 52ms/step
Epoch 109/2000
1/1 - 0s - loss: 2.8526 - val_loss: 2.8877 - 81ms/epoch - 81ms/step
Epoch 110/2000
1/1 - 0s - loss: 2.8513 - val_loss: 2.8866 - 68ms/epoch - 68ms/step
Epoch 111/2000
1/1 - 0s - loss: 2.8500 - val_loss: 2.8856 - 50ms/epoch - 50ms/step
Epoch 112/2000
1/1 - 0s - loss: 2.8488 - val_loss: 2.8845 - 59ms/epoch - 59ms/step
Epoch 113/2000
1/1 

Epoch 200/2000
1/1 - 0s - loss: 2.7386 - val_loss: 2.7796 - 72ms/epoch - 72ms/step
Epoch 201/2000
1/1 - 0s - loss: 2.7375 - val_loss: 2.7786 - 88ms/epoch - 88ms/step
Epoch 202/2000
1/1 - 0s - loss: 2.7364 - val_loss: 2.7775 - 71ms/epoch - 71ms/step
Epoch 203/2000
1/1 - 0s - loss: 2.7354 - val_loss: 2.7765 - 79ms/epoch - 79ms/step
Epoch 204/2000
1/1 - 0s - loss: 2.7343 - val_loss: 2.7754 - 55ms/epoch - 55ms/step
Epoch 205/2000
1/1 - 0s - loss: 2.7332 - val_loss: 2.7744 - 57ms/epoch - 57ms/step
Epoch 206/2000
1/1 - 0s - loss: 2.7321 - val_loss: 2.7734 - 51ms/epoch - 51ms/step
Epoch 207/2000
1/1 - 0s - loss: 2.7311 - val_loss: 2.7725 - 97ms/epoch - 97ms/step
Epoch 208/2000
1/1 - 0s - loss: 2.7300 - val_loss: 2.7717 - 63ms/epoch - 63ms/step
Epoch 209/2000
1/1 - 0s - loss: 2.7289 - val_loss: 2.7709 - 63ms/epoch - 63ms/step
Epoch 210/2000
1/1 - 0s - loss: 2.7278 - val_loss: 2.7699 - 100ms/epoch - 100ms/step
Epoch 211/2000
1/1 - 0s - loss: 2.7268 - val_loss: 2.7689 - 63ms/epoch - 63ms/step
Ep

Epoch 299/2000
1/1 - 0s - loss: 2.6520 - val_loss: 2.7074 - 65ms/epoch - 65ms/step
Epoch 300/2000
1/1 - 0s - loss: 2.6513 - val_loss: 2.7068 - 63ms/epoch - 63ms/step
Epoch 301/2000
1/1 - 0s - loss: 2.6505 - val_loss: 2.7060 - 65ms/epoch - 65ms/step
Epoch 302/2000
1/1 - 0s - loss: 2.6498 - val_loss: 2.7054 - 54ms/epoch - 54ms/step
Epoch 303/2000
1/1 - 0s - loss: 2.6491 - val_loss: 2.7050 - 55ms/epoch - 55ms/step
Epoch 304/2000
1/1 - 0s - loss: 2.6484 - val_loss: 2.7045 - 60ms/epoch - 60ms/step
Epoch 305/2000
1/1 - 0s - loss: 2.6476 - val_loss: 2.7039 - 67ms/epoch - 67ms/step
Epoch 306/2000
1/1 - 0s - loss: 2.6469 - val_loss: 2.7032 - 66ms/epoch - 66ms/step
Epoch 307/2000
1/1 - 0s - loss: 2.6462 - val_loss: 2.7025 - 82ms/epoch - 82ms/step
Epoch 308/2000
1/1 - 0s - loss: 2.6454 - val_loss: 2.7020 - 55ms/epoch - 55ms/step
Epoch 309/2000
1/1 - 0s - loss: 2.6447 - val_loss: 2.7015 - 53ms/epoch - 53ms/step
Epoch 310/2000
1/1 - 0s - loss: 2.6439 - val_loss: 2.7010 - 49ms/epoch - 49ms/step
Epoc

Epoch 398/2000
1/1 - 0s - loss: 2.5743 - val_loss: 2.6349 - 61ms/epoch - 61ms/step
Epoch 399/2000
1/1 - 0s - loss: 2.5737 - val_loss: 2.6363 - 65ms/epoch - 65ms/step
Epoch 400/2000
1/1 - 0s - loss: 2.5732 - val_loss: 2.6339 - 43ms/epoch - 43ms/step
Epoch 401/2000
1/1 - 0s - loss: 2.5730 - val_loss: 2.6353 - 42ms/epoch - 42ms/step
Epoch 402/2000
1/1 - 0s - loss: 2.5719 - val_loss: 2.6321 - 49ms/epoch - 49ms/step
Epoch 403/2000
1/1 - 0s - loss: 2.5706 - val_loss: 2.6318 - 58ms/epoch - 58ms/step
Epoch 404/2000
1/1 - 0s - loss: 2.5697 - val_loss: 2.6322 - 63ms/epoch - 63ms/step
Epoch 405/2000
1/1 - 0s - loss: 2.5691 - val_loss: 2.6307 - 57ms/epoch - 57ms/step
Epoch 406/2000
1/1 - 0s - loss: 2.5690 - val_loss: 2.6320 - 47ms/epoch - 47ms/step
Epoch 407/2000
1/1 - 0s - loss: 2.5679 - val_loss: 2.6288 - 62ms/epoch - 62ms/step
Epoch 408/2000
1/1 - 0s - loss: 2.5668 - val_loss: 2.6283 - 48ms/epoch - 48ms/step
Epoch 409/2000
1/1 - 0s - loss: 2.5661 - val_loss: 2.6304 - 59ms/epoch - 59ms/step
Epoc

Epoch 497/2000
1/1 - 0s - loss: 2.5021 - val_loss: 2.5689 - 48ms/epoch - 48ms/step
Epoch 498/2000
1/1 - 0s - loss: 2.5012 - val_loss: 2.5679 - 46ms/epoch - 46ms/step
Epoch 499/2000
1/1 - 0s - loss: 2.5005 - val_loss: 2.5676 - 64ms/epoch - 64ms/step
Epoch 500/2000
1/1 - 0s - loss: 2.4998 - val_loss: 2.5661 - 42ms/epoch - 42ms/step
Epoch 501/2000
1/1 - 0s - loss: 2.4993 - val_loss: 2.5665 - 41ms/epoch - 41ms/step
Epoch 502/2000
1/1 - 0s - loss: 2.4989 - val_loss: 2.5660 - 46ms/epoch - 46ms/step
Epoch 503/2000
1/1 - 0s - loss: 2.4987 - val_loss: 2.5670 - 46ms/epoch - 46ms/step
Epoch 504/2000
1/1 - 0s - loss: 2.4989 - val_loss: 2.5653 - 50ms/epoch - 50ms/step
Epoch 505/2000
1/1 - 0s - loss: 2.4983 - val_loss: 2.5650 - 43ms/epoch - 43ms/step
Epoch 506/2000
1/1 - 0s - loss: 2.4964 - val_loss: 2.5636 - 47ms/epoch - 47ms/step
Epoch 507/2000
1/1 - 0s - loss: 2.4951 - val_loss: 2.5638 - 41ms/epoch - 41ms/step
Epoch 508/2000
1/1 - 0s - loss: 2.4954 - val_loss: 2.5653 - 48ms/epoch - 48ms/step
Epoc

In [10]:
acc = []
prob = []
x_test_subset = x_masked_test[np.random.choice(x_masked_test.shape[0], size=1000, replace=False)]

for sentence_number in x_test_subset:
    temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array(sentence_number).reshape(1,len(sentence_number)))
    temp = temp[:,-1,:]
    acc.append(1 if temp.argmax() == sentence_number[-1] else 0)
    prob.append(temp[0][sentence_number[-1]])

In [11]:
(np.mean(acc), np.mean(prob))

(0.0, 0.013756235)

In [12]:
embed_dim = 100
num_head = 2

callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
word_embeddings = layers.Embedding(N, embed_dim, name="word_embedding")(inputs)
encoder_output = word_embeddings

for i in range(1):
    encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
mlm_output = layers.Dense(N, name="mlm_cls", activation="softmax")(encoder_output)
mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
adam = Adam()
mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=2000, batch_size=5000, 
                        verbose=2)

Epoch 1/2000
1/1 - 1s - loss: 3.6980 - val_loss: 3.4224 - 1s/epoch - 1s/step
Epoch 2/2000
1/1 - 0s - loss: 3.3798 - val_loss: 3.2408 - 136ms/epoch - 136ms/step
Epoch 3/2000
1/1 - 0s - loss: 3.1896 - val_loss: 3.1299 - 109ms/epoch - 109ms/step
Epoch 4/2000
1/1 - 0s - loss: 3.0727 - val_loss: 3.0596 - 145ms/epoch - 145ms/step
Epoch 5/2000
1/1 - 0s - loss: 2.9989 - val_loss: 3.0090 - 129ms/epoch - 129ms/step
Epoch 6/2000
1/1 - 0s - loss: 2.9465 - val_loss: 2.9705 - 104ms/epoch - 104ms/step
Epoch 7/2000
1/1 - 0s - loss: 2.9068 - val_loss: 2.9417 - 123ms/epoch - 123ms/step
Epoch 8/2000
1/1 - 0s - loss: 2.8766 - val_loss: 2.9195 - 109ms/epoch - 109ms/step
Epoch 9/2000
1/1 - 0s - loss: 2.8526 - val_loss: 2.9008 - 109ms/epoch - 109ms/step
Epoch 10/2000
1/1 - 0s - loss: 2.8317 - val_loss: 2.8837 - 137ms/epoch - 137ms/step
Epoch 11/2000
1/1 - 0s - loss: 2.8122 - val_loss: 2.8683 - 120ms/epoch - 120ms/step
Epoch 12/2000
1/1 - 0s - loss: 2.7943 - val_loss: 2.8546 - 127ms/epoch - 127ms/step
Epoch 1

Epoch 99/2000
1/1 - 0s - loss: 2.4154 - val_loss: 2.6882 - 146ms/epoch - 146ms/step
Epoch 100/2000
1/1 - 0s - loss: 2.4102 - val_loss: 2.6864 - 123ms/epoch - 123ms/step
Epoch 101/2000
1/1 - 0s - loss: 2.4047 - val_loss: 2.6844 - 169ms/epoch - 169ms/step
Epoch 102/2000
1/1 - 0s - loss: 2.3992 - val_loss: 2.6823 - 106ms/epoch - 106ms/step
Epoch 103/2000
1/1 - 0s - loss: 2.3935 - val_loss: 2.6801 - 109ms/epoch - 109ms/step
Epoch 104/2000
1/1 - 0s - loss: 2.3876 - val_loss: 2.6778 - 127ms/epoch - 127ms/step
Epoch 105/2000
1/1 - 0s - loss: 2.3816 - val_loss: 2.6752 - 118ms/epoch - 118ms/step
Epoch 106/2000
1/1 - 0s - loss: 2.3754 - val_loss: 2.6724 - 115ms/epoch - 115ms/step
Epoch 107/2000
1/1 - 0s - loss: 2.3691 - val_loss: 2.6695 - 140ms/epoch - 140ms/step
Epoch 108/2000
1/1 - 0s - loss: 2.3626 - val_loss: 2.6664 - 121ms/epoch - 121ms/step
Epoch 109/2000
1/1 - 0s - loss: 2.3560 - val_loss: 2.6632 - 111ms/epoch - 111ms/step
Epoch 110/2000
1/1 - 0s - loss: 2.3492 - val_loss: 2.6598 - 117ms/

In [13]:
acc = []
prob = []
x_test_subset = x_masked_test[np.random.choice(x_masked_test.shape[0], size=1000, replace=False)]

for sentence_number in x_test_subset:
    temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array(sentence_number).reshape(1,len(sentence_number)))
    temp = temp[:,-1,:]
    acc.append(1 if temp.argmax() == sentence_number[-1] else 0)
    prob.append(temp[0][sentence_number[-1]])

In [14]:
(np.mean(acc), np.mean(prob))

(0.0, 0.0035523782)

In [15]:
x_masked_test

array([[10,  0,  1, 10],
       [10,  0,  2, 10],
       [10,  0,  3, 10],
       ...,
       [19, 18, 15, 19],
       [19, 18, 16, 19],
       [19, 18, 17, 19]])